In [1]:
import pandas as pd
from sklearn.datasets import load_boston
from sklearn.model_selection import train_test_split
from sklearn.linear_model import ElasticNet
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
import numpy as np
import matplotlib.pyplot as plt
import mlflow
import mlflow.sklearn
import shutil  
import os

In [2]:
#Delete last runs directory
try:
    shutil.rmtree('./mlruns')
except FileNotFoundError:
    print("WARNING: Can't find folder mlruns")

In [3]:
data, target = load_boston(return_X_y=True)

df = pd.DataFrame(data)

X_train, X_test, y_train, y_test = train_test_split(df,target,random_state=0)

In [4]:
def calculate_errors(y,ypred):
    mae = mean_absolute_error(y,ypred)
    mse = mean_squared_error(y, ypred)
    R2 = r2_score(y,ypred)
    y,ypred = np.array(y), np.array(ypred)
    mape = np.mean(np.abs((y - ypred) / y)) * 100
#     print("MAE:{0:.3f}, MSE:{1:.2f}, R2:{2:.2f}".format(mae, mse, R2))
    return mae,mse,R2,mape

## Train model

In [5]:
def train_elastic_net(a,l,exp = None):
    with mlflow.start_run(experiment_id=exp): #start mlflow run
        en = ElasticNet(alpha=a,l1_ratio=l)
        en.fit(X_train,y_train)
        y_pred = en.predict(X_test)
        
        #calculate errors
        mae,mse,R2,mape = calculate_errors(y_test,y_pred)
        errors = mae,mse,R2,mape
        print("MAE:{0:.3f}, MSE:{1:.2f}, R2:{2:.2f}".format(mae, mse, R2))
        
        #log metris and parmeters
        mlflow.log_metrics({"MAE":mae,"MSE":mse, "R2":R2, "MAPE":mape})
        mlflow.log_params({"alpha":a,"l1_ratio":l})
        
        #register model
        mlflow.sklearn.log_model(en, "model")
        
        #save error plot
        plt.figure()
        plt.bar(['mae','mse','R2','mape'],errors,color=['blue','red','green','orange']);
        plt.title("Errors")
        plt.savefig("errors.png")
        plt.close()
        mlflow.log_artifact("errors.png")

## Predict with different alpha and l1 ratio values

In [6]:
 for a in np.arange(0.1, 1, 0.25):
        for l in np.arange(0.1, 1, 0.25):
            train_elastic_net(a,l)

MAE:3.787, MSE:32.25, R2:0.61
MAE:3.776, MSE:32.21, R2:0.61
MAE:3.764, MSE:32.16, R2:0.61
MAE:3.749, MSE:32.11, R2:0.61
MAE:3.882, MSE:33.66, R2:0.59
MAE:3.866, MSE:33.58, R2:0.59
MAE:3.845, MSE:33.41, R2:0.59
MAE:3.810, MSE:33.10, R2:0.59
MAE:3.936, MSE:34.57, R2:0.58
MAE:3.929, MSE:34.60, R2:0.58
MAE:3.914, MSE:34.49, R2:0.58
MAE:3.887, MSE:34.24, R2:0.58
MAE:3.966, MSE:35.23, R2:0.57
MAE:3.968, MSE:35.38, R2:0.57
MAE:3.970, MSE:35.47, R2:0.57
MAE:3.968, MSE:35.53, R2:0.57


In [7]:
print("target mean:{}".format(target.mean()))
print("target std:{}".format(target.std()))

target mean:22.532806324110677
target std:9.188011545278203


## new experiment

In [8]:
exp = mlflow.create_experiment(name="normalized data")

In [9]:
from sklearn import preprocessing

In [10]:
data = preprocessing.normalize(data)
df = pd.DataFrame(data)

X_train, X_test, y_train, y_test = train_test_split(df,target,random_state=0)

In [11]:
data.shape

(506, 13)

In [12]:
 for a in np.arange(0.1, 1, 0.25):
        for l in np.arange(0.1, 1, 0.25):
            train_elastic_net(a,l,exp)

MAE:5.762, MSE:72.00, R2:0.12
MAE:5.731, MSE:71.32, R2:0.13
MAE:5.698, MSE:70.70, R2:0.13
MAE:5.723, MSE:70.57, R2:0.14
MAE:6.041, MSE:76.93, R2:0.06
MAE:6.015, MSE:76.32, R2:0.07
MAE:5.959, MSE:75.21, R2:0.08
MAE:5.821, MSE:72.91, R2:0.11
MAE:6.121, MSE:78.68, R2:0.04
MAE:6.111, MSE:78.46, R2:0.04
MAE:6.089, MSE:77.95, R2:0.05
MAE:6.026, MSE:76.54, R2:0.06
MAE:6.163, MSE:79.55, R2:0.03
MAE:6.165, MSE:79.58, R2:0.03
MAE:6.166, MSE:79.59, R2:0.03
MAE:6.166, MSE:79.59, R2:0.03


# Find best model

In [13]:
df = mlflow.search_runs()
run_id = df.loc[df['metrics.MAE'].idxmin()]['run_id']
print("Minimum error run_id: ",run_id)

Minimum error run_id:  365c2acb8adf49baa3029c45d70eec2c


In [14]:
df

,run_id,experiment_id,status,artifact_uri,start_time,end_time,metrics.MAE,metrics.MAPE,metrics.R2,metrics.MSE,params.alpha,params.l1_ratio,tags.mlflow.user,tags.mlflow.source.name,tags.mlflow.source.type,tags.mlflow.log-model.history
0,a5b8204a5fa347a88f5ff2efea4aa517,0,FINISHED,file:///C:/ie/session2/mlruns/0/a5b8204a5fa347...,2021-05-26 22:30:52.581000+00:00,2021-05-26 22:30:52.778000+00:00,3.968131,18.322409,0.565145,35.527237,0.8499999999999999,0.8499999999999999,jicav,C:\Users\jicav\anaconda3\lib\site-packages\ipy...,LOCAL,"[{""run_id"": ""a5b8204a5fa347a88f5ff2efea4aa517""..."
1,73b9f35af14f4c03ac32b425c15c0c52,0,FINISHED,file:///C:/ie/session2/mlruns/0/73b9f35af14f4c...,2021-05-26 22:30:52.307000+00:00,2021-05-26 22:30:52.565000+00:00,3.969933,18.344260,0.565791,35.474462,0.8499999999999999,0.6,jicav,C:\Users\jicav\anaconda3\lib\site-packages\ipy...,LOCAL,"[{""run_id"": ""73b9f35af14f4c03ac32b425c15c0c52""..."
2,bd294aca3e224adfb263b3afc28505ca,0,FINISHED,file:///C:/ie/session2/mlruns/0/bd294aca3e224a...,2021-05-26 22:30:52.107000+00:00,2021-05-26 22:30:52.307000+00:00,3.968318,18.353080,0.566991,35.376397,0.8499999999999999,0.35,jicav,C:\Users\jicav\anaconda3\lib\site-packages\ipy...,LOCAL,"[{""run_id"": ""bd294aca3e224adfb263b3afc28505ca""..."
3,c445b18096ba478e851e45d8b32728fd,0,FINISHED,file:///C:/ie/session2/mlruns/0/c445b18096ba47...,2021-05-26 22:30:51.906000+00:00,2021-05-26 22:30:52.107000+00:00,3.966483,18.365372,0.568795,35.229018,0.8499999999999999,0.1,jicav,C:\Users\jicav\anaconda3\lib\site-packages\ipy...,LOCAL,"[{""run_id"": ""c445b18096ba478e851e45d8b32728fd""..."
4,ea254acf053447fe95c7a719c9c81c28,0,FINISHED,file:///C:/ie/session2/mlruns/0/ea254acf053447...,2021-05-26 22:30:51.709000+00:00,2021-05-26 22:30:51.906000+00:00,3.886942,18.029630,0.580933,34.237367,0.6,0.8499999999999999,jicav,C:\Users\jicav\anaconda3\lib\site-packages\ipy...,LOCAL,"[{""run_id"": ""ea254acf053447fe95c7a719c9c81c28""..."
5,907a39f7baee40e586ca5fa8a622f1b2,0,FINISHED,file:///C:/ie/session2/mlruns/0/907a39f7baee40...,2021-05-26 22:30:51.478000+00:00,2021-05-26 22:30:51.706000+00:00,3.914044,18.155179,0.577847,34.489518,0.6,0.6,jicav,C:\Users\jicav\anaconda3\lib\site-packages\ipy...,LOCAL,"[{""run_id"": ""907a39f7baee40e586ca5fa8a622f1b2""..."
6,ce9e5c0fb5ed4b8e81bfdf8ee224839d,0,FINISHED,file:///C:/ie/session2/mlruns/0/ce9e5c0fb5ed4b...,2021-05-26 22:30:51.275000+00:00,2021-05-26 22:30:51.478000+00:00,3.928626,18.228895,0.576530,34.597062,0.6,0.35,jicav,C:\Users\jicav\anaconda3\lib\site-packages\ipy...,LOCAL,"[{""run_id"": ""ce9e5c0fb5ed4b8e81bfdf8ee224839d""..."
7,48bf9bea15914d428f6a130dbcff0dc6,0,FINISHED,file:///C:/ie/session2/mlruns/0/48bf9bea15914d...,2021-05-26 22:30:51.028000+00:00,2021-05-26 22:30:51.265000+00:00,3.935648,18.274140,0.576849,34.571040,0.6,0.1,jicav,C:\Users\jicav\anaconda3\lib\site-packages\ipy...,LOCAL,"[{""run_id"": ""48bf9bea15914d428f6a130dbcff0dc6""..."
8,3c4e27bc19ac4148b7321f4bac53b2a1,0,FINISHED,file:///C:/ie/session2/mlruns/0/3c4e27bc19ac41...,2021-05-26 22:30:50.813000+00:00,2021-05-26 22:30:51.028000+00:00,3.809643,17.773695,0.594847,33.100608,0.35,0.8499999999999999,jicav,C:\Users\jicav\anaconda3\lib\site-packages\ipy...,LOCAL,"[{""run_id"": ""3c4e27bc19ac4148b7321f4bac53b2a1""..."
9,3c2b083f760048ccac6a34413c0f8f3e,0,FINISHED,file:///C:/ie/session2/mlruns/0/3c2b083f760048...,2021-05-26 22:30:50.594000+00:00,2021-05-26 22:30:50.813000+00:00,3.844676,17.913881,0.591077,33.408624,0.35,0.6,jicav,C:\Users\jicav\anaconda3\lib\site-packages\ipy...,LOCAL,"[{""run_id"": ""3c2b083f760048ccac6a34413c0f8f3e""..."


## We can serve the model as a REST API 
`mlflow models serve -m [model_path] -p [port]`

## And then call the API to make a prediction for us 


In [13]:
#first we need to turn one of the test rows into json format
X_test.loc[[329]].to_json(orient='split')

'{"columns":[0,1,2,3,4,5,6,7,8,9,10,11,12],"index":[329],"data":[[0.0001176914,0.0,0.0056710311,0.0,0.0008051464,0.0110847655,0.0301054738,0.0091272095,0.007001273,0.7526368457,0.0295803784,0.656736909,0.0128473359]]}'

And then we are ready to call the API service and get a reply  
`curl http://127.0.0.1:2323/invocations -H 'Content-Type: application/json' -d '{"columns":[0,1,2,3,4,5,6,7,8,9,10,11,12],"data":[[0.06724,0.0,3.24,0.0,0.46,6.333,17.2,5.2146,4.0,430.0,16.9,375.21,7.34]]}'`

## We can also use mlflow to make a prediction out of a csv or json file
First, we need to have the data we want to predict in csv format


In [14]:
X_test.to_csv (r'data.csv', index = False, header=True)

`mlflow models predict -m mlruns/1/281e22a41418413c8e0ca608aeb78289/artifacts/model/ -i data.csv  -t csv`